<a href="https://colab.research.google.com/github/youngbin03/Tiny-NeRF/blob/main/nerf_fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeRF-WandB-COLAB

### [NeRF 공식 저장소](https://github.com/bmild/nerf)를 수정하여 [wandb 연동이 편리하도록 만든 저장소](https://github.com/ProtossDragoon/nerf-wandb.git)의 소스코드를 이용해 커스텀 데이터셋에 NeRF 모델 학습

Plank-ing Hyundong 3D Reconstruction Project
Created 2023.01.12 <br>

## 환경

### GPU 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

### TF 런타임 변경 (python 다운그레이드 -> 3.6)

In [ ]:
!add-apt-repository ppa:deadsnakes/ppa
!apt-get update
!apt-get install python3.6
!apt-get install python3.6-dev

!wget https://bootstrap.pypa.io/get-pip.py && python3.6 get-pip.py

import sys

sys.path[2] = '/usr/lib/python36.zip'
sys.path[3] = '/usr/lib/python3.6'
sys.path[4] = '/usr/lib/python3.6/lib-dynload'
sys.path[5] = '/usr/local/lib/python3.6/dist-packages'
sys.path[7] ='/usr/local/lib/python3.6/dist-packages/IPython/extensions'

In [ ]:
try:
    %tensorflow_version 1.x
except ValueError:
    # 만약 %tensorflow_version 1.x magic 명령어가 작동하지 않는 경우
    !pip uninstall --yes tensorflow
    !pip install tensorflow==1.15
    import tensorflow
    print(tensorflow.__version__)

### 의존 패키지 설치

In [ ]:
!apt-get update
!sudo apt -qq install imagemagick
!pip install ConfigArgParse -qqq
!pip install imageio-ffmpeg -qqq

### NeRF 소스코드 다운로드

In [ ]:
!git clone https://github.com/ProtossDragoon/nerf-wandb.git

In [ ]:
%cd nerf-wandb
#!git checkout dev # 개발 중
!ls -al

### Weight and Bias 설치

- 실험 관리 도구로 wandb 를 사용합니다.

In [ ]:
!pip install wandb -qqq

In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

## 커스텀 데이터로 실행

- LLFF 를 통해 얻은 pose 가 필요합니다.

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

### 파라미터 설정

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%y%m%d_%H%M%S')

dataset_name = 'hyundong360_50' #@param ['hyundong360_50', 'hyundong360_100', 'hyundong360_200', 'hyundong360_400', 'noparking360_100', 'noparking360_200', 'noparking360_400', 'noparking720_100', 'noparking720_200', 'noparking720_400', 'hyundong360_100_background', 'hyundong360_100_cleanup', 'hyundong360_50_cleanup', 'hyundong360_25_cleanup']
downsample_factor = 4 #@param {type:"slider", min:1, max:64, step:1}
netdepth = 6 #@param {type:"slider", min:4, max:16, step:2}
netwidth = 96 #@param {type:"slider", min:64, max:256, step:4}
experiment_name = f'{dataset_name}_{downsample_factor}_downsampled_{now}'
max_iter = 30000 #@param
learning_rate = 0.001 #@param
video_saving_cnt = 3 #@param {type:"slider", min:1, max:10, step:1}
n_samples = 128 #@param {type:"slider", min:32, max:256, step:32}

# fine 모델에서 사용되는 샘플 개수는 coarse 모델의 sampling 개수의 2배로 설정한다.
# 공식 논문에서 제안하는 대로, 64이면 128.
n_importance = n_samples * 2

# Reproduce 를 위해 고정 random_seed 를 사용
random_seed = 128 #@param

# tradeoff: memory <-> speed (training 에는 속도와 성능 모두에 영향을 미치지 않음. 학습 도중 동영상을 만들 때 OOM 이 난다면 충분히 낮출 것)
rendering_speed = 2048 #@param {type:"slider", min:1024, max:16384, step:1024}

# tradeoff: memory <-> result
n_points_per_ray = 65536 #@param {type:"slider", min:2048, max:262144, step:1024}

_dummy_dir = '/content/drive/MyDrive/fire/data/logs/{experiment_name}'
_tensorboard_logdir = '/content/drive/MyDrive/fire/data/logs/summaries/{experiment_name}'
print(f'experiment: {experiment_name}')

Mesh 코드를 실행하기 위한 config.txt 생성

In [ ]:
# Make config.txt file.
f = open("/content/drive/MyDrive/fire/data/logs/config.txt", 'w')
def make_config(a, b):

  if b is None:
    f.write(a + '\n')
  else:
    data = a + ' = ' + b + '\n'
    f.write(data)


make_config("maxiter", str(max_iter))
make_config("datadir", '/content/drive/MyDrive/fire/data')
make_config("basedir", '/content/drive/MyDrive/fire/data/logs')
make_config("dataset_type", 'llff')
make_config('factor', str(downsample_factor))
make_config('netdepth', str(netdepth))
make_config('netwidth', str(netwidth))
make_config('netdepth_fine', str(netdepth))
make_config('netwidth_fine', str(netwidth))
make_config('chunk', str(rendering_speed))
make_config('netchunk', str(n_points_per_ray))
make_config('lrate', str(learning_rate))
make_config('i_video', str(max_iter // video_saving_cnt))
make_config('expname', experiment_name)
make_config('N_samples', str(n_samples))
make_config('N_importance', str(n_importance))
make_config('random_seed', str(random_seed))
make_config('raw_noise_std', '1.0')
make_config('use_viewdirs', None)
make_config('no_ndc', None)
make_config('spherify', None)
make_config('lindisp', None)

f.close()

### 학습 시작

- 360도에 걸쳐 촬영한 데이터의 경우, `--no_ndc`, `--spherify`, `--lindisp` 를 추가하세요.

In [ ]:
!python run_nerf.py \
    --wandbproject {'fire-sinyoungbin0128'} \
    --wandbentity {'sinyoungbin0128'} \
    --maxiter {max_iter} \
    --datadir /content/drive/MyDrive/fire/data \
    --basedir /content/drive/MyDrive/fire/data/logs \
    --dataset_type llff \
    --factor {downsample_factor} \
    --netdepth {netdepth} \
    --netwidth {netwidth} \
    --netdepth_fine {netdepth} \
    --netwidth_fine {netwidth} \
    --chunk {rendering_speed} \
    --netchunk {n_points_per_ray} \
    --lrate {learning_rate} \
    --i_video {max_iter // video_saving_cnt} \
    --expname {experiment_name} \
    --N_samples {n_samples} \
    --N_importance {n_importance} \
    --random_seed {random_seed} \
    --raw_noise_std 1.0 \
    --use_viewdirs \
    --no_ndc \
    --spherify \
    --lindisp